# **Практика 8: Методы защиты от атак на модели ИИ**

Выполнил студент 2 курса группы: **ББМО-02-23**

ФИО: **Курченко Иван Дмитриевич**

# **Цель задания:**
Изучить методы защиты моделей ИИ от различных атак, включая методы защиты на уровне данных,моделирования и обучения. Реализовать эти методы и проверить их эффективность против атак, изученных ранее.

##**Шаги выполнения:**

### **Шаг 1: Защита с помощью Adversarial Training**

 **Adversarial Training** — это метод защиты, который заключается в том, чтобы обучать модель на противоречивых примерах. Этот метод помогает модели научиться быть более устойчивой к атакам, так как она сталкивается с противоречивыми примерами на этапе обучения.

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

# Уменьшаем количество обучающих изображений до 1000
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image

# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)

        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)

        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adversarial_image.reshape(28, 28))

    return np.array(adversarial_images)

# Создание модели
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
    for epoch in range(5):
        print(f"Epoch {epoch + 1}/{5}")
        for i in range(0, len(train_images), 32):
            batch_images = train_images[i:i + 32]
            batch_labels = train_labels[i:i + 32]

            # Генерация противоречивых примеров для текущей партии данных
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)

            # Объединение оригинальных и противоречивых примеров
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)

            # Обучение на комбинированных данных
            model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()

# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели
model.save('Adversarial_model.h5')

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


###**Шаг 2: Градиентная маскировка (Gradient Masking)**

 **Gradient Masking** — это метод защиты, который затрудняет доступ к градиентам модели для атак. Он используется для уменьшения информации, доступной для атакующих, и усложнения поиска направленных изменений.

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
# Загрузка и подготовка данных:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных:
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding:
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)
# Обновление модели для градиентной маскировки:
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
# Обучение модели с градиентной маскировкой:
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с градиентной маскировкой:
masked_model.save('Gradient_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8717 - loss: 0.4719
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9620 - loss: 0.1285
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9745 - loss: 0.0857
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9796 - loss: 0.0643
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9855 - loss: 0.0495


###**Шаг 3: Регуляризация и нормализация для повышения устойчивости**

Использование таких методов, как **L2-регуляризация**, **дропаут** и **нормализация батчей**, может помочь улучшить устойчивость модели к атакам.

In [7]:
# Модель с регуляризацией и нормализацией:
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model
# Обучение модели с регуляризацией и нормализацией:
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией:
regularized_model.save('Regularized_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.7928 - loss: 1.3243
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.8694 - loss: 0.6288
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8665 - loss: 0.6271
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8679 - loss: 0.6145
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8635 - loss: 0.6269


###**Шаг 4: Оценка моделей на противоречивых примерах**

Теперь проверим эффективность всех защитных методов на атакованных данных, созданных с помощью **FGSM** и других методов, таких как **PGD** или **GAN**.

In [8]:
# Загрузка атакованной модели:
protected_model = tf.keras.models.load_model('Adversarial_model.h5')
# Генерация противоречивых примеров для тестовых данных:
adversarial_test_images = generate_adversarial_examples(protected_model, test_images, test_labels, epsilon=0.1)
# Оценка модели с Adversarial Training:
print("Adversarially Trained Model Accuracy on Adversarial Examples:")
adv_loss, adv_acc = protected_model.evaluate(adversarial_test_images, test_labels, verbose=1)
# Оценка модели с Gradient Masking:
print("Masked Model Accuracy on Adversarial Examples:")
masked_loss, masked_acc = masked_model.evaluate(adversarial_test_images, test_labels, verbose=1)
# Оценка модели с Регуляризацией и нормализацией:
print("Regularized Model Accuracy on Adversarial Examples:")
reg_loss, reg_acc = regularized_model.evaluate(adversarial_test_images, test_labels, verbose=1)

Adversarially Trained Model Accuracy on Adversarial Examples:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4935 - loss: 1.4711
Masked Model Accuracy on Adversarial Examples:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6907 - loss: 0.9749
Regularized Model Accuracy on Adversarial Examples:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6742 - loss: 1.1294


### **Результаты**

Adversarial Training (Точность: 49.35%, Потери: 1.4711): Модель, обученная на противоречивых примерах, демонстрирует умеренную устойчивость к FGSM-атакам. Хотя её точность выше, чем у модели с градиентной маскировкой, потери остаются высокими, что указывает на эффективность Adversarial Training, но также на его ограниченность в отношении более сложных или неизвестных атак.


Masked Model (Точность: 69.07%, Потери: 0.9749): Модель с градиентной маскировкой показывает более высокую точность и более низкие потери по сравнению с Adversarial Training. Это подтверждает, что Gradient Masking усложняет создание атак, но не устраняет их полностью. Метод может оказаться уязвимым к адаптивным атакам, которые могут обойти градиентные ограничения.
Regularized Model (Точность: 67.42%, Потери: 1.1294): Модель с регуляризацией достигает хорошей точности среди всех методов защиты, однако её потери остаются на высоком уровне. Это свидетельствует о её способности лучше классифицировать атакованные примеры, но за счет высоких потерь. Данный метод может быть полезен в ситуациях, где критична точность, но не требуется строгое подавление атак.

**Итоговые выводы**

Лучший результат по точности: Модель с градиентной маскировкой достигает 69.07%. Тем не менее, высокие потери указывают на то, что модель не полностью устойчива к FGSM-атакам.

Умеренная устойчивость: Adversarial Training предлагает сбалансированную защиту с точностью 49.35% и умеренными потерями.

Ограниченная эффективность: Gradient Masking демонстрирует лучшую точность (69.07%) и более низкие потери (0.9749) по сравнению с Adversarial Training.
Эти результаты подчеркивают важность выбора правильного метода защиты в зависимости от требований к точности и устойчивости к атакам в различных приложениях машинного обучения.